<a href="https://colab.research.google.com/github/ag4267research1/Solving-Poisson-s-Equation-with-GNO/blob/main/GNOPDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install neuraloperator --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.6/248.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 112.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 72.7 MB/s eta 0:00:00


In [91]:
import os, pickle
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import inspect
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

# Correct imports:
from neuralop.layers.gno_block import GNOBlock
from neuralop.data.transforms.normalizers import UnitGaussianNormalizer
from neuralop.training import Trainer



In [5]:
# Folder in the Colab VM (not Drive)
!mkdir -p /content/nonlinear_poisson
!cd /content/nonlinear_poisson

# Download with progress bar (percentage + speed)
!wget --show-progress \
  "https://zenodo.org/records/15001788/files/nonlinear_poisson.obj?download=1" \
  -O nonlinear_poisson.obj




--2025-12-07 18:59:03--  https://zenodo.org/records/15001788/files/nonlinear_poisson.obj?download=1
Resolving zenodo.org (zenodo.org)... 188.185.48.75, 137.138.52.235, 188.185.43.153, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9608410281 (8.9G) [application/octet-stream]
Saving to: ‘nonlinear_poisson.obj’

nonlinear_poisson.o 100%[===================>]   8.95G  19.1MB/s    in 8m 2s   

2025-12-07 19:07:05 (19.0 MB/s) - ‘nonlinear_poisson.obj’ saved [9608410281/9608410281]



In [12]:
# Show file size
!ls -lh nonlinear_poisson.obj

-rw-r--r-- 1 root root 9.0G Dec  7 19:07 nonlinear_poisson.obj


In [15]:
DATA_PATH = "/content/nonlinear_poisson.obj"
print("Exists?", os.path.exists(DATA_PATH))

Exists? True


In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

DATA_PATH = "/content/nonlinear_poisson.obj"
print("Exists?", os.path.exists(DATA_PATH))

with open(DATA_PATH, "rb") as f:
    raw_data = pickle.load(f)

print("Type:", type(raw_data))
print("Number of PDE samples:", len(raw_data))

sample = raw_data[0]
print("Sample keys:", sample.keys())

Device: cuda
Exists? True
Type: <class 'list'>
Number of PDE samples: 10000
Sample keys: dict_keys(['train_points_boundary', 'train_values_boundary', 'train_source_terms_boundary', 'train_bc_boundary', 'train_points_domain', 'train_values_domain', 'train_distances_domain', 'train_source_terms_domain', 'train_bc_domain', 'val_points_boundary', 'val_values_boundary', 'val_source_terms_boundary', 'val_points_domain', 'val_values_domain', 'val_source_terms_domain', 'coefs'])


In [38]:

def flatten_coefs_dict(coefs_dict):
    out_list = []
    for k, v in coefs_dict.items():
        if isinstance(v, np.ndarray):          # numpy array
            out_list.extend(v.reshape(-1).tolist())
        elif np.isscalar(v):                   # single float
            out_list.append(float(v))
        else:
            # Unexpected type: ignore or print for debugging
            # print("Skipping key:", k, "value type:", type(v))
            continue
    return torch.tensor(out_list, dtype=torch.float32)


class NonlinearPoissonDataset(Dataset):
    def __init__(self, data_list, split="train"):
        self.data_list = data_list
        self.split = split

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        item = self.data_list[idx]

        # 1. Coordinates (P,2)
        coords = torch.tensor(item[f"{self.split}_points_domain"], dtype=torch.float32)

        # 2. Forcing (P,) → (P,1)
        f = torch.tensor(item[f"{self.split}_source_terms_domain"], dtype=torch.float32).unsqueeze(-1)

        # 3. Distance to boundary (P,) → (P,1)
        d = torch.tensor(item[f"{self.split}_distances_domain"], dtype=torch.float32).unsqueeze(-1)

        # 4. Flatten geometry coefficients
        coefs = flatten_coefs_dict(item["coefs"])            # (C,)
        coefs = coefs.unsqueeze(0).expand(coords.shape[0], -1)  # (P, C)

        # 5. Stack all features → input X
        features = torch.cat([f, d, coefs], dim=-1)          # (P, 1 + 1 + C)

        # 6. Solution u (P,) → (P,1)
        u = torch.tensor(item[f"{self.split}_values_domain"], dtype=torch.float32).unsqueeze(-1)

        return coords, features, u

In [39]:
print(type(sample["coefs"]))
print(sample["coefs"])


<class 'dict'>
{'seed': 1, 'c1': np.float32(-0.0007273823), 'c2': np.float32(-0.0042264136), 'r0': 1.0, 'beta': array([-1.1116347 ,  0.66393006], dtype=float32), 'mu_1': array([-0.84975487, -1.102674  ], dtype=float32), 'mu_2': array([ 0.9603709, -1.4707267], dtype=float32), 'b': array([ 0.5957165 ,  0.4740579 , -0.05061221, -0.35185003,  0.6810765 ],
      dtype=float32)}


In [40]:
coords0, feats0, u0 = next(iter(train_loader))
print(coords0.shape, feats0.shape, u0.shape)

torch.Size([4, 10000, 2]) torch.Size([4, 10000, 17]) torch.Size([4, 10000, 1])


In [41]:
# Simple 90/10 split
n_total = len(raw_data)
n_train = int(0.9 * n_total)
train_raw = raw_data[:n_train]
val_raw   = raw_data[n_train:]

train_ds = NonlinearPoissonDataset(train_raw, split="train")
val_ds   = NonlinearPoissonDataset(val_raw,   split="val")

def collate_fn(batch):
    """
    Because every PDE has the same number of points,
    default collate would actually work, but this keeps it explicit.
    """
    coords, feats, u = zip(*batch)  # each is list length B

    coords = torch.stack(coords, dim=0)  # (B, P, 2)
    feats  = torch.stack(feats,  dim=0)  # (B, P, C_in)
    u      = torch.stack(u,      dim=0)  # (B, P, 1)

    return coords, feats, u

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True,
                          num_workers=2, collate_fn=collate_fn)
val_loader   = DataLoader(val_ds,   batch_size=4, shuffle=False,
                          num_workers=2, collate_fn=collate_fn)

coords0, feats0, u0 = next(iter(train_loader))
print("coords batch:", coords0.shape)
print("features batch:", feats0.shape)
print("u batch:", u0.shape)

coords batch: torch.Size([4, 10000, 2])
features batch: torch.Size([4, 10000, 17])
u batch: torch.Size([4, 10000, 1])


# MODEL

In [102]:
class GNOPoissonModel(nn.Module):
    def __init__(
        self,
        in_channels=17,
        hidden_channels=64,
        out_channels=1,
        n_layers=4,
        radius=0.1,
        coord_dim=2
    ):
        super().__init__()

        self.layers = nn.ModuleList()

        # First GNO layer
        self.layers.append(
            GNOBlock(
                in_channels=in_channels,
                out_channels=hidden_channels,
                coord_dim=coord_dim,
                radius=radius,
                use_open3d_neighbor_search=False
            )
        )

        # Middle
        for _ in range(n_layers - 2):
            self.layers.append(
                GNOBlock(
                    in_channels=hidden_channels,
                    out_channels=hidden_channels,
                    coord_dim=coord_dim,
                    radius=radius,
                    use_open3d_neighbor_search=False
                )
            )

        # Last
        self.layers.append(
            GNOBlock(
                in_channels=hidden_channels,
                out_channels=out_channels,
                coord_dim=coord_dim,
                radius=radius,
                use_open3d_neighbor_search=False
            )
        )

        self.head = nn.Sequential(
            nn.Linear(out_channels, 32),
            nn.ReLU(),
            nn.Linear(32, out_channels)
        )

    def forward(self, coords, feats):
        """
        coords: (B, N, 2)
        feats:  (B, N, C)
        """

        # Fix: enforce 3D tensors
        if coords.ndim == 2:
            coords = coords.unsqueeze(0)
        if feats.ndim == 2:
            feats = feats.unsqueeze(0)

        B, N, _ = coords.shape

        # GNO expects no batch mixing inside neighbor search.
        # So iterate over each batch independently.
        outputs = []

        for b in range(B):
            y = coords[b]   # (N, 2)
            x = coords[b]   # (N, 2)
            h = feats[b]    # (N, C)

            for layer in self.layers:
                h = layer(y, x, h)

            outputs.append(self.head(h))

        return torch.stack(outputs, dim=0)  # (B, N, out_channels)


In [78]:
def forward(self, coords, feats):
    """
    coords: (B, N, 2)
    feats:  (B, N, C)
    """

    # If missing batch dimension → add it
    if coords.ndim == 2:
        coords = coords.unsqueeze(0)
    if feats.ndim == 2:
        feats = feats.unsqueeze(0)

    # GNO expects y, x, f_y shape = (B, N, *)
    y = coords
    x = coords
    h = feats

    for layer in self.layers:
        h = layer(y, x, h)

    return self.head(h)

In [103]:
# Build model
in_channels = feats0.shape[-1]
print("Input feature channels:", in_channels)

model = GNOPoissonModel(
    in_channels=in_channels,
    hidden_channels=64,
    coord_dim=2,
    radius=0.2,
    n_layers=3,
).to(device)

print(model)



Input feature channels: 17
GNOPoissonModel(
  (layers): ModuleList(
    (0-1): 2 x GNOBlock(
      (pos_embedding): SinusoidalEmbedding()
      (neighbor_search): NeighborSearch()
      (integral_transform): IntegralTransform(
        (channel_mlp): LinearChannelMLP(
          (fcs): ModuleList(
            (0): Linear(in_features=256, out_features=128, bias=True)
            (1): Linear(in_features=128, out_features=256, bias=True)
            (2): Linear(in_features=256, out_features=128, bias=True)
            (3): Linear(in_features=128, out_features=64, bias=True)
          )
        )
      )
    )
    (2): GNOBlock(
      (pos_embedding): SinusoidalEmbedding()
      (neighbor_search): NeighborSearch()
      (integral_transform): IntegralTransform(
        (channel_mlp): LinearChannelMLP(
          (fcs): ModuleList(
            (0): Linear(in_features=256, out_features=128, bias=True)
            (1): Linear(in_features=128, out_features=256, bias=True)
            (2): Linear(i

In [104]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
criterion = nn.MSELoss()


In [105]:
coords0, feats0, u0 = next(iter(train_loader))

print("coords0", type(coords0), coords0.shape)
print("feats0", type(feats0))
print("u0", type(u0))


coords0 <class 'torch.Tensor'> torch.Size([4, 10000, 2])
feats0 <class 'torch.Tensor'>
u0 <class 'torch.Tensor'>


In [106]:
print("GNOBlock.forward signature:\n")
print(inspect.signature(GNOBlock.forward))

GNOBlock.forward signature:

(self, y, x, f_y=None)


In [107]:
print(inspect.getsource(GNOBlock.forward))


    def forward(self, y, x, f_y=None):
        """Compute a GNO neighbor search and kernel integral transform.

        Parameters
        ----------
        y : torch.Tensor of shape [n, d1]
            n points of dimension d1 specifying
            the space to integrate over.
            If batched, these must remain constant
            over the whole batch so no batch dim is needed.
        x : torch.Tensor of shape [m, d1], default None
            m points of dimension d1 over which the
            output function is defined. Must share domain
            with y
        f_y : torch.Tensor of shape [batch, n, d2] or [n, d2], default None
            Function to integrate the kernel against defined
            on the points y. The kernel is assumed diagonal
            hence its output shape must be d3 for the transforms
            (b) or (d). If None, (a) is computed.

        Output
        ----------
        out_features : torch.Tensor of shape [batch, m, d3] or [m, d3]
     

In [108]:
print(inspect.signature(GNOBlock.forward))
print(inspect.getsource(GNOBlock.forward))


(self, y, x, f_y=None)
    def forward(self, y, x, f_y=None):
        """Compute a GNO neighbor search and kernel integral transform.

        Parameters
        ----------
        y : torch.Tensor of shape [n, d1]
            n points of dimension d1 specifying
            the space to integrate over.
            If batched, these must remain constant
            over the whole batch so no batch dim is needed.
        x : torch.Tensor of shape [m, d1], default None
            m points of dimension d1 over which the
            output function is defined. Must share domain
            with y
        f_y : torch.Tensor of shape [batch, n, d2] or [n, d2], default None
            Function to integrate the kernel against defined
            on the points y. The kernel is assumed diagonal
            hence its output shape must be d3 for the transforms
            (b) or (d). If None, (a) is computed.

        Output
        ----------
        out_features : torch.Tensor of shape [batch, 

In [109]:
def run_epoch(loader, train=True):
    if train:
        model.train()
    else:
        model.eval()

    epoch_loss = 0.0
    n_batches = 0

    # tqdm progress bar over batches
    pbar = tqdm(loader, desc="Train" if train else "Val", leave=False)

    for coords, feats, u in pbar:
        coords = coords.to(device)      # (B, N, 2)
        feats  = feats.to(device)       # (B, N, C)
        u      = u.to(device)           # (B, N, 1)

        if train:
            optimizer.zero_grad()

        with torch.set_grad_enabled(train):
            u_pred = model(coords, feats)          # (B, N, 1)
            loss = criterion(u_pred, u)

            if train:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()

        epoch_loss += loss.item()
        n_batches += 1
        pbar.set_postfix({"loss": f"{epoch_loss / n_batches:.3e}"})

    return epoch_loss / max(1, n_batches)


n_epochs = 20  # bump this up once everything is stable
train_losses = []
val_losses = []

epoch_bar = tqdm(range(1, n_epochs + 1), desc="Epochs")

for epoch in epoch_bar:
    train_loss = run_epoch(train_loader, train=True)
    val_loss   = run_epoch(val_loader,   train=False)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    epoch_bar.set_postfix({
        "train": f"{train_loss:.3e}",
        "val":   f"{val_loss:.3e}",
    })

    print(f"[Epoch {epoch:03d}] train MSE: {train_loss:.4e}   val MSE: {val_loss:.4e}")


Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Train:   0%|          | 0/2250 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.45 GiB. GPU 0 has a total capacity of 22.16 GiB of which 1.71 GiB is free. Process 39107 has 20.44 GiB memory in use. Of the allocated memory 20.03 GiB is allocated by PyTorch, and 198.10 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)